In [305]:
import geopandas as gpd
from tqdm import tqdm
import libpysal as lp
import json

### Get the data in the folder ./LCPS_data for pre-processing. Let's call this `retrieved data`.

In [259]:
def retrieve_data(sy):
    """
    Retrieve data for a given school year (sy)
    """
    data_dir = "./LCPS_data"
    # Read the data files
    schools = gpd.read_file('{}/LCPS_Sites_{}.shp'.format(data_dir, sy))
    students = gpd.read_file('{}/Students_{}.shp'.format(data_dir, sy))
    spas = gpd.read_file('{}/PlanningZones_{}.shp'.format(data_dir, sy))
    
    return schools, students, spas

In [260]:
sy = '2017_2018'

In [261]:
schools, students, spas = retrieve_data(sy)

#### Print out a data instance of the SPAs, SCHOOLs and STUDENTs

In [262]:
for i, spa in spas.iterrows():
    print(spa)
    break

OBJECTID                                                      1
COUNT_                                                        5
STDYAREA                                                 WL03.4
ELEM_                                                       126
INT_                                                        205
HIGH_                                                       311
ELEM_CODE                                                   LOV
INT_CODE                                                    HRM
HIGH_CODE                                                   WHS
DISTRICT                                                     WL
UTILITIES                                                   WOU
SHAPE_Leng                                              47268.8
SHAPE_Area                                          6.39938e+07
STDYAREA_1                                               WL03.4
DISTRICT_1                                                   WL
PLANNING_Z                              

In [263]:
for ind, school in schools.iterrows():
    print(school)
    break

OBJECTID                                                1
SCH_CODE                                              HCA
CLASS                                             CHARTER
SCH_NUM                                               119
NAME                            HILLSBORO CHARTER ACADEMY
DATE_OPENE                                           1966
BUILDING_D                                              7
CLASSROOMS                                              6
SPECIAL_SI                                              0
TRAILERS                                                0
SCHL_CODE                                             119
STRT_GRD                                               -1
END_GRD                                                 5
ELEM_                                                 119
INT_                                                    0
MID_                                                    0
HIGH_                                                   0
CAPACITY      

In [264]:
for index, student in students.iterrows():
    print(student)
    break

ObjectID                                                1
ARC_Single                     21365 FITZGERALD DR, 20147
Loud_ID                                            100011
Address                                             21365
Address_Ex                                           None
ADDwEXT                                             21365
Prefix                                               None
Street_Nam                                     FITZGERALD
Street_Typ                                             DR
Suffix                                               None
Town                                              ASHBURN
Zip_1                                               20147
Subdivisio                                  FARMWELL HUNT
GRID_CODE                                          100011
GRADE                                                  13
IEP_FLAG                                                N
GENDER                                                  M
ETHNIC        

### Printout the CRS of the SPAs, SCHOOLs and STUDENTs

In [265]:
spas.crs    # Printout the CRS

{'proj': 'lcc',
 'lat_1': 38.03333333333333,
 'lat_2': 39.2,
 'lat_0': 37.66666666666666,
 'lon_0': -78.5,
 'x_0': 3499999.999999998,
 'y_0': 2000000,
 'ellps': 'GRS80',
 'towgs84': '0,0,0,0,0,0,0',
 'units': 'us-ft',
 'no_defs': True}

In [266]:
spas.keys()

Index(['OBJECTID', 'COUNT_', 'STDYAREA', 'ELEM_', 'INT_', 'HIGH_', 'ELEM_CODE',
       'INT_CODE', 'HIGH_CODE', 'DISTRICT', 'UTILITIES', 'SHAPE_Leng',
       'SHAPE_Area', 'STDYAREA_1', 'DISTRICT_1', 'PLANNING_Z', 'ELEM', 'MID',
       'HIGH', 'UTILITIE_1', 'DISTRICT_2', 'MID_2018', 'HIGH_2018', 'PK', 'KG',
       'GR1', 'GR2', 'GR3', 'GR4', 'GR5', 'GR6', 'GR7', 'GR8', 'GR9', 'GR10',
       'GR11', 'GR12', 'TOTAL_KG_5', 'TOTAL_6_8', 'TOTAL_9_12', 'TOTAL_KG_1',
       'TOTAL_PK_1', 'ELEM_1', 'MID_1', 'HIGH_1', 'DISTRICT_3', 'MID_2_2019',
       'HIGH__2019', 'PLANNING_1', 'UTILITIE_2', 'STUDENTS_P', 'Field39',
       'Field40', 'geometry'],
      dtype='object')

In [267]:
[x for x in list(spas.PLANNING_1) if x.startswith('DS07')]

['DS07.2',
 'DS07.7',
 'DS07.6',
 'DS07.5',
 'DS07.8',
 'DS07.1',
 'DS07',
 'DS07.9',
 'DS07.3',
 'DS07.4',
 'DS07.11']

In [268]:
students.crs

{'proj': 'lcc',
 'lat_1': 38.03333333333333,
 'lat_2': 39.2,
 'lat_0': 37.66666666666666,
 'lon_0': -78.5,
 'x_0': 3499999.999999998,
 'y_0': 2000000,
 'ellps': 'GRS80',
 'towgs84': '0,0,0,0,0,0,0',
 'units': 'us-ft',
 'no_defs': True}

In [269]:
schools.crs

{'proj': 'lcc',
 'lat_1': 38.03333333333333,
 'lat_2': 39.2,
 'lat_0': 37.66666666666666,
 'lon_0': -78.5,
 'x_0': 3499999.999999998,
 'y_0': 2000000,
 'ellps': 'GRS80',
 'towgs84': '0,0,0,0,0,0,0',
 'units': 'us-ft',
 'no_defs': True}

### Read the data (supplied with the algorithm) in the folder './data'. Let's call this `new data`.

In [270]:
spas_new = gpd.read_file('./data/SPAs.json')

In [271]:
[x for x in list(spas_new.SPA) if x.startswith('DS07')]

['DS07.2',
 'DS07.12',
 'DS07.7',
 'DS07.6',
 'DS07.5',
 'DS07.8',
 'DS07.1',
 'DS07',
 'DS07.9',
 'DS07.3',
 'DS07.4',
 'DS07.11']

In [272]:
for i, s in spas_new.iterrows():
    print(s)
    break

OBJECTID                                                      1
COUNT_                                                        5
SPA                                                      WL03.4
ELEM_                                                       126
INT_                                                        205
HIGH_                                                       311
ELEM_CODE                                                   LOV
ELEM_POP                                                     31
MID_CODE                                                    HRM
MID_POP                                                      12
HIGH_CODE                                                   WHS
HIGH_POP                                                     13
DISTRICT                                                     WL
UTILITIES                                                   WOU
SHAPE_Leng                                              47261.2
SHAPE_Area                              

#### We have to modify the SPAs in `retrieved data` to match the entries in `new data`

In [273]:
fields = ['ELEM_1', 'MID_1', 'HIGH_1', 'DISTRICT_3', 'MID_2_2019', 'HIGH__2019',
          'PLANNING_1', 'UTILITIE_2', 'STUDENTS_P', 'Field39', 'Field40']
spas.drop(fields, axis=1, inplace=True)

In [276]:
# Adding new fields and renaming some existing ones
spas = spas.rename(columns={"INT_CODE": "MID_CODE"})
spas = spas.rename(columns={"STDYAREA": "SPA"})
new_fields = ['ELEM_POP', 'MID_POP', 'HIGH_POP', 'TOT_POP']    # Population of ELEM, MID, HIGH and TOTAL

for f in new_fields:
    spas[f] = 0

##### Do point-in-polygon test to find out number of students attending public schools in LCPS and residing within Loudoun county, VA.

In [277]:
spas.columns

Index(['OBJECTID', 'COUNT_', 'SPA', 'ELEM_', 'INT_', 'HIGH_', 'ELEM_CODE',
       'MID_CODE', 'HIGH_CODE', 'DISTRICT', 'UTILITIES', 'SHAPE_Leng',
       'SHAPE_Area', 'STDYAREA_1', 'DISTRICT_1', 'PLANNING_Z', 'ELEM', 'MID',
       'HIGH', 'UTILITIE_1', 'DISTRICT_2', 'MID_2018', 'HIGH_2018', 'PK', 'KG',
       'GR1', 'GR2', 'GR3', 'GR4', 'GR5', 'GR6', 'GR7', 'GR8', 'GR9', 'GR10',
       'GR11', 'GR12', 'TOTAL_KG_5', 'TOTAL_6_8', 'TOTAL_9_12', 'TOTAL_KG_1',
       'TOTAL_PK_1', 'geometry', 'ELEM_POP', 'MID_POP', 'HIGH_POP', 'TOT_POP'],
      dtype='object')

In [278]:
# spas['SPA'] = spas['STDYAREA_1']

In [279]:
set(students['GRADE'])    # Possible values of grade

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}

In [280]:
count = 0
others = 0
for index, student in tqdm(students.iterrows()):
    student_location = student.geometry
    student_grade = student['GRADE']
    legit = True
    for ind, spa in spas.iterrows():
        
        if student_location.within(spa.geometry):
            count += 1
            if 0 < student_grade < 6 or student_grade == 13:    # ELEMENTARY School has grades KG-5
                spas.at[ind, 'ELEM_POP'] = spas['ELEM_POP'][ind] + 1
            elif 5 < student_grade < 9:    # MIDDLE School has grades 6-8
                spas.at[ind, 'MID_POP'] = spas['MID_POP'][ind] + 1
            elif 8 < student_grade < 13:    # HIGH School has grades 9-12
                spas.at[ind, 'HIGH_POP'] = spas['HIGH_POP'][ind] + 1
            else:
                legit = False
                others += 1
#                 print(others)
                
            if legit:
                spas.at[ind, 'TOT_POP'] = spas['TOT_POP'][ind] + 1
                
            break
            
total_students = len(students)


81427it [30:20, 44.74it/s]


In [281]:
print('{}/{} students living inside LCPS\' boundary are considered for redistricting'.format(count - others,
                                                                                             total_students))
print('{}/{} students living inside LCPS\' boundary are ignored'.format(others, total_students))
print('{}/{} students living outside LCPS\' boundary are ignored'.format(total_students - count,
                                                                         total_students))

80345/81427 students living inside LCPS' boundary are considered for redistricting
1054/81427 students living inside LCPS' boundary are ignored
28/81427 students living outside LCPS' boundary are ignored


In [294]:
count = 0
others = 0
for index, school in tqdm(schools.iterrows()):
    school_location = school.geometry
    legit = True
    for ind, spa in spas.iterrows():
        
        if school_location.within(spa.geometry):
            count += 1
            if spa['STDYAREA_1']:
                print(spa['STDYAREA_1'])
                schools.at[index, 'SPA'] = str(spa['STDYAREA_1'])
            else:
                legit = False
                others += 1
                
            if legit:
                pass
                
            break
#         assert 0, 'not contained'
            


8it [00:00, 36.78it/s]

WL53
DN46.1
EL05
DS13
CL27
DS12.3
WL38
DN30
CL27


13it [00:00, 39.42it/s]

EL03
EL03
EL27
EL27
CL37
DN36.3
DS13.1
WL31


26it [00:00, 44.30it/s]

...

CL07
DN08.5
DN51
DN47
DN47
DN36
DN04
DS11
CL07.2


99it [00:02, 42.75it/s]

DS07.5
DN23
DN08.4
DS12.5
DS07.5
CL25.1
DN01.4
DS07.11
DN36.3
DN36.3


105it [00:02, 42.88it/s]

CL34.1
DS07.3
DS07.3


In [295]:
len(spas)

446

In [284]:
schools['SCHOOL_TYP'] = ''

schools.loc[schools['CLASS'] == 'ELEMENTARY','SCHOOL_TYP'] = 'ES'

schools.loc[schools['CLASS'] == 'MIDDLE','SCHOOL_TYP'] = 'MS'

schools.loc[schools['CLASS'] == 'HIGH','SCHOOL_TYP'] = 'HS'

In [285]:
spas.keys()

Index(['OBJECTID', 'COUNT_', 'SPA', 'ELEM_', 'INT_', 'HIGH_', 'ELEM_CODE',
       'MID_CODE', 'HIGH_CODE', 'DISTRICT', 'UTILITIES', 'SHAPE_Leng',
       'SHAPE_Area', 'STDYAREA_1', 'DISTRICT_1', 'PLANNING_Z', 'ELEM', 'MID',
       'HIGH', 'UTILITIE_1', 'DISTRICT_2', 'MID_2018', 'HIGH_2018', 'PK', 'KG',
       'GR1', 'GR2', 'GR3', 'GR4', 'GR5', 'GR6', 'GR7', 'GR8', 'GR9', 'GR10',
       'GR11', 'GR12', 'TOTAL_KG_5', 'TOTAL_6_8', 'TOTAL_9_12', 'TOTAL_KG_1',
       'TOTAL_PK_1', 'geometry', 'ELEM_POP', 'MID_POP', 'HIGH_POP', 'TOT_POP'],
      dtype='object')

### Transform the CRS of the `retrieved data` to match `new data`. We need to project the shapefiles into a new coordinate system to ensure that the data you are working with uses common geometric projection. For more details refer to the [link](https://geopandas.org/projections.html).

In [286]:
new_crs = spas_new.crs

In [287]:
new_crs

{'init': 'epsg:2924'}

In [288]:
spas = spas.to_crs(new_crs)

/home/jeffchan/.vre/p36/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [289]:
schools['SCHOOL_TYP'].fillna('')

0        
1      ES
2      ES
3      ES
4      HS
5      ES
6      ES
7      ES
8      HS
9      ES
10     MS
11     HS
12     MS
13     MS
14     MS
15     ES
16     ES
17     ES
18     ES
19     HS
20     ES
21     ES
22     ES
23       
24     MS
25     HS
26     MS
27     ES
28     MS
29       
       ..
75     ES
76     ES
77     MS
78     ES
79     ES
80     MS
81     ES
82       
83       
84     ES
85     MS
86     HS
87     ES
88     ES
89     ES
90       
91     HS
92     HS
93     ES
94     MS
95     ES
96       
97     ES
98     HS
99       
100    ES
101      
102      
103      
104      
Name: SCHOOL_TYP, Length: 105, dtype: object

In [290]:
schools = schools.to_crs(new_crs)

/home/jeffchan/.vre/p36/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [291]:
# Check if the projections have been reprojected 
print(spas.crs, schools.crs)

{'init': 'epsg:2924'} {'init': 'epsg:2924'}


In [303]:
spas.keys()

Index(['OBJECTID', 'COUNT_', 'SPA', 'ELEM_', 'INT_', 'HIGH_', 'ELEM_CODE',
       'MID_CODE', 'HIGH_CODE', 'DISTRICT', 'UTILITIES', 'SHAPE_Leng',
       'SHAPE_Area', 'STDYAREA_1', 'DISTRICT_1', 'PLANNING_Z', 'ELEM', 'MID',
       'HIGH', 'UTILITIE_1', 'DISTRICT_2', 'MID_2018', 'HIGH_2018', 'PK', 'KG',
       'GR1', 'GR2', 'GR3', 'GR4', 'GR5', 'GR6', 'GR7', 'GR8', 'GR9', 'GR10',
       'GR11', 'GR12', 'TOTAL_KG_5', 'TOTAL_6_8', 'TOTAL_9_12', 'TOTAL_KG_1',
       'TOTAL_PK_1', 'geometry', 'ELEM_POP', 'MID_POP', 'HIGH_POP', 'TOT_POP'],
      dtype='object')

In [304]:
adjacency_matrix = dict(lp.weights.Rook.from_dataframe(spas, idVariable="STDYAREA_1"))
for key in adjacency_matrix:
    adjacency_matrix[key] = list(adjacency_matrix[key].keys())
with open("data/nbrlist_SPA.json", "w") as fp:
    json.dump(adjacency_matrix, fp)

### Write out the updated data as geojson files

In [292]:
def write_data(schools, spas, sy):
    """
    Retrieve data for a given school year (sy)
    """
    data_dir = "./LCPS_data"
    # Read the data files
    schools.to_file('{}/Schools_{}.json'.format(data_dir, sy), driver='GeoJSON')
    spas.to_file('{}/SPAs_{}.json'.format(data_dir, sy), driver='GeoJSON')


In [306]:
write_data(schools, spas, sy)